In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import helpers
%autosave 0

Autosave disabled


# Modern CNN Architectures

## Loading TensorBoard Graphs for pre-built models

Inside of the `prebuilt` folder, there are TensorBoard graphs exported for VGGNet, InceptionV1, and ResNet models. You will use these as guidance for creating your own layer functions. Load them up in TensorBoard by using the following command (assuming you're running this command from the `notebooks` directory:

```shell
tensorboard --logdir=prebuilt
```

Navigate to `localhost:6006` in your browser. After you click on the "Graphs" link, you'll be able to switch to the various reference graphs by choosing from the dropdown "runs" option.

![](images/06/tb1.png)

![](images/06/tb2.png)

Below is a description of each graph:

* **inception_module**: This is a graph representing a single InceptionV1 module. 
* **inceptionv1**: The entire InceptionV1 network (without the "auxiliary training branch")
* **residual_stack**: A set of three bottleneck residual blocks. Blocks 1 and 2 are identical, while `Block_0` showcases two nuances of the model:
    * Using a pooling layer to downsample the inputs, labeled as "downsample"
    * Using a $1\times1$ convolution to adjust the input channel depth to match the output, labeled as "shortcut"
* **resnet_152**: The entire ResNet network (152-layer version)
* **vgg_19**: The entire VGGNet network (19-layer version)

The goal of this notebook/lab is to recreate the above graphs in TensorFlow.

### Provided Layer functions

#### `conv()`

Creates a 2D convolutional layer with Xavier-initialized weights. Automatically detects depth from previous layer

* **Arguments**
    * `inputs`: 4D `Tensor` with shape `[batch_size, height, width, channels]`
    * `depth`: The number of output channels this convolution should create. Scalar number.
    * `ksize`: 2D list of integers. The dimensions of convolutional kernel (ie. [3,3], [5,5], etc)
    * `strides`: 2D list of integers. The strides of the convolution (defaults to `[1, 1]`)
    * `padding`: String, accepted values `'SAME'` or `'VALID'`. The type of padding to use. Defaults to `SAME`.
    * `bval`: Floating point number. The initial values for biases
    * `activation_fn`: Lambda function. The activation function to use. defaults to `tf.nn.relu`
    * `scope`: The name to use for the variable scope.
    
* **Returns**
    * A 4D `Tensor` after the convolution operation.

In [2]:
def conv(inputs, depth, ksize, strides=[1, 1], padding='SAME',
         bval=0.01, activation_fn=tf.nn.relu, scope=None):
    prev_shape = inputs.get_shape().as_list()
    prev_depth = prev_shape[-1]
    kshape = ksize + [prev_depth, depth]
    strides = [1] + strides + [1]
    fan_in = np.prod(prev_shape[1:], dtype=np.float32)
    with tf.variable_scope(scope, 'conv_layer'):
        xavier_stddev = tf.sqrt(tf.constant(2.0, dtype=tf.float32) / fan_in, name='xavier_stddev')
        w = tf.Variable(tf.truncated_normal(kshape, stddev=xavier_stddev), name='kernel')
        conv = tf.nn.conv2d(inputs, w, strides, padding, name='conv')
        if bval:
            b = tf.Variable(tf.constant(bval, shape=[depth]), name='bias')
            z = tf.nn.bias_add(conv, b)
        return z if activation_fn is None else activation_fn(z)

#### `fully_connected()`

Creates a 2D fully connected layer with Xavier-initialized weights. Automatically detects depth from previous layer.

* **Arguments**
    * `inputs`: 2D `Tensor` with shape `[batch_size, depth]`
    * `depth`: Scalar. The number of neurons in this layer
    * `bval`: Floating point number. The initial values for biases
    * `activation_fn`: Lambda function. The activation function to use. defaults to `tf.nn.relu`
    * `keep_prob`: Scalar float indicating the keep probability for dropout (if any)
    * `scope`: The name to use for the variable scope.
    
* **Returns**
    * A 2D `Tensor` 

In [3]:
def fully_connected_layer(inputs, depth, bval=0.01, activation_fn=tf.nn.relu, 
                          keep_prob=None, scope=None):
    inputs = tf.convert_to_tensor(inputs)
    prev_shape = inputs.get_shape().as_list()
    fan_in = prev_shape[-1]
    with tf.variable_scope(scope, 'fully_connected'):
        xavier_stddev = tf.sqrt(tf.constant(2.0, dtype=tf.float32) / fan_in, name='xavier_stddev')
        w = tf.Variable(tf.truncated_normal([fan_in, depth], stddev=xavier_stddev), name='W')
        b = tf.Variable(tf.constant(bval, shape=[depth]), name='bias')
        z = tf.matmul(inputs, w) + b
        a = z if activation_fn is None else activation_fn(z)
        return a if keep_prob is None else tf.nn.dropout(a, keep_prob)

#### `avgpool()` and `maxpool()`

Performs average pooling and max pooling, respectively

* **Arguments**
    * `inputs`: 4D `Tensor` with shape `[batch_size, height, width, channels]`
    * `ksize`: 2D list of integers. The dimensions of pooling kernel (ie. [2,2] etc)
    * `strides`: 2D list of integers. The strides of the pooling kernel.
    * `padding`: String, accepted values `'SAME'` or `'VALID'`. The type of padding to use. Defaults to `VALID`.
    * `name`: The name to use for the variable scope.
    
* **Returns**
    * A 4D `Tensor` after the pooling operation.

In [4]:
def avgpool(inputs, ksize, strides, padding='VALID', name=None):
    with tf.name_scope(name, 'avgpool'):
        ksize = [1] + ksize + [1]
        strides = [1] + strides + [1]
        return tf.nn.avg_pool(inputs, ksize, strides, padding)

    
def maxpool(inputs, ksize, strides, padding='VALID', name=None):
    with tf.name_scope(name, 'maxpool'):
        ksize = [1] + ksize + [1]
        strides = [1] + strides + [1]
        return tf.nn.max_pool(inputs, ksize, strides, padding)

#### `flatten()`

Flattens an N dimensional `Tensor` into a 2D `Tensor` (ie from shape `[batch_size, a, b, c]` to shape `[batch_size, a*b*c]`

* **Arguments**
    * `inputs`: The input `Tensor` to flatten
    * `scope`: The name to use for the variable scope.
    
* **Returns**
    * A 2D flattened `Tensor`

In [5]:
def flatten(inputs, name=None):
    prev_shape = inputs.get_shape().as_list()
    fan_in = np.prod(prev_shape[1:])
    with tf.name_scope(name, 'flatten'):
        return tf.reshape(inputs, [-1, fan_in])

## VGGNet

[VGGNet paper on arXiv.org](https://arxiv.org/abs/1409.1556)

![](images/06/vggtable.png)

Use the above layer functions to recreate the 19 layer VGGNet from the above table (column E). Your model function should expect two parameter inputs:

* `inputs`: a 4D tensor with dtype `float32` and shape `[batch_size, 224, 224, 3]`
* `keep_prob`: A scalar `Tensor` with dtype `float32` representing the keep_probability for dropout

In [6]:
def vggnet(inputs, keep_prob):
    a = inputs
    with tf.name_scope('conv1'):
        for i in range(2):
            a = conv(a, 64, [3, 3])
    a = maxpool(a, [2, 2], [2, 2])
    with tf.name_scope('conv2'):
        for i in range(2):
            a = conv(a, 128, [3, 3])
    a = maxpool(a, [2, 2], [2, 2])
    with tf.name_scope('conv3'):
        for i in range(4):
            a = conv(a, 256, [3, 3])
    a = maxpool(a, [2, 2], [2, 2])
    with tf.name_scope('conv4'):
        for i in range(4):
            a = conv(a, 512, [3, 3])
    a = maxpool(a, [2, 2], [2, 2])
    with tf.name_scope('conv5'):
        for i in range(4):
            a = conv(a, 512, [3, 3])
    a = maxpool(a, [2, 2], [2, 2])
    a = flatten(a)
    a = fully_connected_layer(a, 4096)
    a = tf.nn.dropout(a, keep_prob)
    a = fully_connected_layer(a, 4096)
    a = tf.nn.dropout(a, keep_prob)
    a = fully_connected_layer(a, 1000, activation_fn=None)
    return tf.nn.softmax(a)

In [7]:
# Test module: Run once you're ready to check your work
graph = tf.Graph()
with graph.as_default():
    inputs = tf.random_normal([10, 224, 224, 3])
    keep_prob = tf.placeholder(tf.float32)
    output = vggnet(inputs, keep_prob)
    writer = tf.summary.FileWriter('tbout/vggnet', graph=graph)
    writer.close()

Run TensorBoard to check your work

```shell
tensorboard --logdir=tbout/vggnet
```

## Inception-v1

[Inception paper on arXiv.org](https://arxiv.org/abs/1409.4842)

### Define an Inception module layer function

Create a function that generates the following graph (from the paper):

![](images/06/inception2.png)

Your function should have the following parameters:

* `inputs`: 4D `Tensor` inputs, with shape `[batch_size, height, width, channels]` 
* `depth_1x1`: Scalar int. The number of output channels from the `1x1` branch of the module.
* `reduce_3x3`: Scalar int. The number of channels the `1x1` convolution on the `3x3` branch should output into the `3x3` convolution.
* `depth_3x3`: Scalar int. The number of output channels from the `3x3` branch of the module.
* `reduce_5x5`: Scalar int. The number of channels the `1x1` convolution on the `5x5` branch should output into the `5x5` convolution.
* `depth_5x5`: Scalar int. The number of output channels from the `5x5` branch of the module.
* `pool_proj`: Scalar int. The number of output channels the `1x1` convolution on the max pool branch
* `scope`: Optional string. A name for the module. (Don't need to worry about this for implementation)

Here are some additional hints about creating the module:

* The max pooling operation needs to be `3x3` with stride 1
* The above function parameters follows Table 1 from the paper (see below for embedded table in notebook) 
* You'll need to use [`tf.concat()`](https://www.tensorflow.org/api_docs/python/tf/concat) to attach all of the outputs from each branch to one another. `tf.concat` takes two parameters. The first is a list of `Tensor` objects to concatenate, and the other is the axis (or dimension) they should be attached. <br> For example, if you have two tensors, `a` and `b`, with shape `[10, 2, 2, 5]`, here are the results of different concat axes:

In [8]:
a = tf.zeros([10, 2, 2, 5])
b = tf.ones([10, 2, 2, 5])

with tf.Session() as sess:
    print('Axis = 0: {}'.format(tf.concat([a, b], 0).get_shape()))
    print('Axis = 1: {}'.format(tf.concat([a, b], 1).get_shape()))
    print('Axis = 2: {}'.format(tf.concat([a, b], 2).get_shape()))
    print('Axis = 3: {}'.format(tf.concat([a, b], 3).get_shape()))

Axis = 0: (20, 2, 2, 5)
Axis = 1: (10, 4, 2, 5)
Axis = 2: (10, 2, 4, 5)
Axis = 3: (10, 2, 2, 10)


It's your job to determine which axis is appropriate for the goal of concatenating the filters of each branch.

In [9]:
def inception_module(inputs, depth_1x1, reduce_3x3, depth_3x3, reduce_5x5,
                     depth_5x5, pool_proj, scope=None):
    with tf.variable_scope(scope, 'inception_module') as scope:
        with tf.variable_scope('Branch_0'):
            # 1x1 convolution path
            relu_0 = conv(inputs=inputs,
                          ksize=[1, 1],
                          depth=depth_1x1,
                          strides=[1, 1],
                          padding='SAME',
                          scope='conv_1x1')
        with tf.variable_scope('Branch_1'):
            # 3x3 convolution path
            relu_1 = conv(inputs=inputs,
                          ksize=[1, 1],
                          depth=reduce_3x3,
                          strides=[1, 1],
                          padding='SAME',
                          scope='conv_1x1')
            relu_1 = conv(inputs=relu_1,
                          ksize=[3, 3],
                          depth=depth_3x3,
                          strides=[1, 1],
                          padding='SAME',
                          scope='conv_3x3')
        with tf.variable_scope('Branch_2',):
            # 5x5 convolution path
            relu_2 = conv(inputs=inputs,
                          ksize=[1, 1],
                          depth=reduce_5x5,
                          strides=[1, 1],
                          padding='SAME',
                          scope='conv_1x1')
            relu_2 = conv(inputs=relu_2,
                          ksize=[5, 5],
                          depth=depth_5x5,
                          strides=[1, 1],
                          padding='SAME',
                          scope='conv_5x5')
        with tf.variable_scope('Branch_3'):
            pool = maxpool(inputs=inputs,
                           ksize=[3, 3],
                           strides=[1, 1],
                           padding='SAME',
                           name='maxpool')
            relu_3 = conv(inputs=pool,
                          ksize=[1, 1],
                          depth=pool_proj,
                          strides=[1, 1],
                          padding='SAME',
                          scope='conv_1x1')
        return tf.concat([relu_0, relu_1, relu_2, relu_3], 3)

In [10]:
# Test module: Run once you're ready to check your work
graph = tf.Graph()
with graph.as_default():
    inputs = tf.random_normal([10, 28, 28, 192])
    output = inception_module(inputs, 64, 96, 128, 16, 32, 32, 'inception_module')
    writer = tf.summary.FileWriter('tbout/inception_module', graph=graph)
    writer.close()

Run TensorBoard to check your work

```shell
tensorboard --logdir=tbout/inception_module
```

### Creating the full Inception-v1 network

![](images/06/inceptiontable.png)

Once your Inception module is functioning correctly, try to create the inception module described in the above model. Remember that the inception_module function parameters are designed to line up with the "#1x1", "#3x3 reduce", etc columns.

In [11]:
def inceptionv1(inputs, keep_prob):
    with tf.variable_scope('inceptionv1'):
        c = conv(inputs, 64, [7, 7], [2, 2], scope='conv_7x7_stride_2')
        p = maxpool(c, [3, 3], [2, 2], name='maxpool')
        c = conv(p, 192, [3, 3], scope='conv_3x3')
        p = maxpool(c, [3, 3], [2, 2], 'SAME', name='maxpool')                   
        i_3a = inception_module(inputs=p,
                                depth_1x1=64,
                                reduce_3x3=96,
                                depth_3x3=128,
                                reduce_5x5=16,
                                depth_5x5=32,
                                pool_proj=32,
                                scope='Mixed_3a')
        i_3b = inception_module(inputs=i_3a,
                                depth_1x1=128,
                                reduce_3x3=128,
                                depth_3x3=192,
                                reduce_5x5=32,
                                depth_5x5=96,
                                pool_proj=64,
                                scope='Mixed_3b')
        p = maxpool(i_3b, [3, 3], [2, 2], 'SAME', name='maxpool')
        i_4a = inception_module(inputs=p,
                                depth_1x1=192,
                                reduce_3x3=96,
                                depth_3x3=208,
                                reduce_5x5=16,
                                depth_5x5=48,
                                pool_proj=64,
                                scope='Mixed_4a')
        i_4b = inception_module(inputs=i_4a,
                                depth_1x1=160,
                                reduce_3x3=112,
                                depth_3x3=224,
                                reduce_5x5=24,
                                depth_5x5=64,
                                pool_proj=64,
                                scope='Mixed_4b')
        i_4c = inception_module(inputs=i_4b,
                                depth_1x1=128,
                                reduce_3x3=128,
                                depth_3x3=256,
                                reduce_5x5=24,
                                depth_5x5=64,
                                pool_proj=64,
                                scope='Mixed_4c')
        i_4d = inception_module(inputs=i_4c,
                                depth_1x1=112,
                                reduce_3x3=144,
                                depth_3x3=288,
                                reduce_5x5=32,
                                depth_5x5=64,
                                pool_proj=64,
                                scope='Mixed_4d')
        i_4e = inception_module(inputs=i_4d,
                                depth_1x1=256,
                                reduce_3x3=160,
                                depth_3x3=320,
                                reduce_5x5=32,
                                depth_5x5=128,
                                pool_proj=128,
                                scope='Mixed_4e')
        p = maxpool(i_4e, [3, 3], [2, 2], 'SAME', name='maxpool')
        i_5a = inception_module(inputs=p,
                                depth_1x1=256,
                                reduce_3x3=160,
                                depth_3x3=320,
                                reduce_5x5=32,
                                depth_5x5=128,
                                pool_proj=128,
                                scope='Mixed_5a')
        i_5b = inception_module(inputs=i_5a,
                                depth_1x1=384,
                                reduce_3x3=192,
                                depth_3x3=384,
                                reduce_5x5=48,
                                depth_5x5=128,
                                pool_proj=128,
                                scope='Mixed_5b')
        p = avgpool(i_5b, [7, 7], [1, 1], 'VALID', name='avgpool')
        dropped = tf.nn.dropout(p, keep_prob=keep_prob)
        flat = flatten(dropped)
        fc = fully_connected_layer(flat, 1000, activation_fn=None, scope='linear')
        softmax = tf.nn.softmax(fc)
        return softmax

In [12]:
# Test module: Run once you're ready to check your work
graph = tf.Graph()
with graph.as_default():
    inputs = tf.random_normal([10, 224, 224, 3])
    keep_prob = tf.placeholder(tf.float32)
    output = inceptionv1(inputs, keep_prob)
    writer = tf.summary.FileWriter('tbout/inceptionv1', graph=graph)
    writer.close()

Run TensorBoard to check your work

```shell
tensorboard --logdir=tbout/inceptionv1
```

## ResNet

[ResNet paper on arXiv.org](https://arxiv.org/abs/1512.03385)

### Define a residual block layer function

Create a function that recreates a "bottleneck" residual block:

![](images/06/resnet4.png)

Your function, `residual_block` should take the following arguments:

* `inputs`: A 4D Tensor with shape `[batch_size, height, width, channels]`
* `bottleneck_depth`: Scalar int. Number of channels the first `1x1` and `3x3` convolutions should output. (e.g. the number 64 in the image above)
* `output_depth`: Scalar int. Number of channels the final `1x1` convolution should output (e.g. the number 256 in the image above)
* `downsample`: Boolean, defaults to False. If True, the function should add a maxpool (2x2, stride 2) operation before the first `1x1` convolution
* `scope`: Optional string. Name for the residual block.

Notes for implementing the residual block:

* **Important**: Your function must check to see if the `output_depth` matches the number of channels in `inputs`. If it does not, the "skip connection" needs to have a `1x1` convolution applied to it to adjust the number of channels (i.e. `skip = conv(inputs, output_depth, [1,1],[1,1])`
* **Important**: Don't use any activation function for the output of the final `1x1` convolution. Instead, apply the activation function _after_ you add in the skip connection.

In [13]:
def residual_block(inputs, bottleneck_depth, output_depth, downsample=False, scope=None):
    with tf.variable_scope(scope, 'residual_block'):
        if downsample:
            inputs = maxpool(inputs=inputs,
                           ksize=[2, 2],
                           strides=[2, 2],
                           padding='VALID',
                           name='downsample')
        prev_depth = inputs.get_shape()[3]
        relu1 = conv(inputs=inputs,
                     depth=bottleneck_depth,
                     ksize=[1, 1],
                     strides=[1, 1],
                     padding='SAME',
                     scope='conv1')
        relu2 = conv(inputs=relu1,
                     depth=bottleneck_depth,
                     ksize=[3, 3],
                     strides=[1, 1],
                     padding='SAME',
                     scope='conv2')
        conv3 = conv(inputs=relu2,
                     depth=output_depth,
                     ksize=[1, 1],
                     strides=[1, 1],
                     padding='SAME',
                     scope='conv3',
                     activation_fn=None)
        if inputs.get_shape() != conv3.get_shape():
            inputs = conv(inputs=inputs,
                        depth=output_depth,
                        ksize=[1, 1],
                        strides=[1, 1],
                        padding='SAME',
                        scope='shortcut',
                        activation_fn=None)
        add = tf.add(conv3, inputs)
        return tf.nn.relu(add)

In [14]:
# Test
graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder(tf.float32, shape=[10, 224, 224, 3], name='inputs')
    block = inputs
    for i in range(3):
        ds = True if i == 0 else False  # down-sample first block only
        block = residual_block(inputs=block,
                               bottleneck_depth=128,
                               output_depth=512,
                               scope='block_{}'.format(i),
                               downsample=ds)
    writer = tf.summary.FileWriter('tbout/residual_stack', graph=graph)

Run TensorBoard to check your work

```shell
tensorboard --logdir=tbout/residual_stack
```

### Creating the full ResNet

![](images/06/resnettable.png)

Use your `residual_block` function to implement the 152-layer ResNet from the above table. 

Notes:

* In between each layer section ("conv2_x", "conv3_x", etc), there is a 2x2 maxpool, stride 2.
* Use the `for` loop in the `residual_block` test code above as a model for creating the different layer sections.


In [15]:
def resnet(inputs, keep_prob):
    input_depth = inputs.get_shape()[3]
    c = conv(inputs=inputs, depth=64, ksize=[7, 7], strides=[2, 2], padding='SAME', scope='conv1')
    p = maxpool(inputs=c, ksize=[3, 3], strides=[2, 2], padding='SAME', name='maxpool_3x3')
    block = p  # makes below loops more semantic
    with tf.variable_scope('stack_1'):
        for i in range(3):
            block = residual_block(inputs=block, bottleneck_depth=64, output_depth=256, scope='block_{}'.format(i))
    with tf.variable_scope('stack_2'):
        for i in range(8):
            ds = True if i == 0 else False  # down-sample first block only
            block = residual_block(inputs=block, bottleneck_depth=128, output_depth=512, 
                                   scope='block_{}'.format(i), downsample=ds)
    with tf.variable_scope('stack_3'):
        for i in range(36):
            ds = True if i == 0 else False  # down-sample first block only
            block = residual_block(inputs=block, bottleneck_depth=256, output_depth=1024,
                                          scope='block_{}'.format(i), downsample=ds)
    with tf.variable_scope('stack_4'):
        for i in range(3):
            ds = True if i == 0 else False  # down-sample first block only
            block = residual_block(inputs=block, bottleneck_depth=512, output_depth=2048,
                                          scope='block_{}'.format(i), downsample=ds)
    p = avgpool(inputs=block, ksize=[7, 7], strides=[1, 1], padding='VALID', name='avgpool_7x7')
    flat = flatten(p)
    fc = fully_connected_layer(flat, 1000, activation_fn=None, scope='linear')    
    softmax = tf.nn.softmax(fc)
    return softmax

In [16]:
# Test module: Run once you're ready to check your work
graph = tf.Graph()
with graph.as_default():
    inputs = tf.random_normal([10, 224, 224, 3])
    keep_prob = tf.placeholder(tf.float32)
    output = resnet(inputs, keep_prob)
    writer = tf.summary.FileWriter('tbout/resnet', graph=graph)
    writer.close()

## Inception-Resnet-v2 Implementation

In [17]:
# Question: are we using an activation function? It's not specified so I am assuming relu as default for now
# except for the 1x1 convs they said had no activation - also is no bias correct - because doing batch norm?
# Inception-Resnet-v2 stem
def inception_stem(inputs):
    with tf.variable_scope('stem') as scope:
        with tf.variable_scope("3x3_1"):
            a = conv(inputs, depth=32, ksize=[3,3], strides=[2,2], padding='VALID', bval=None, activation_fn=tf.nn.relu)
        with tf.variable_scope("3x3_2"):
            # not sure about the strides here?
            a = conv(a, depth = 32, ksize=[3,3], strides=[2,2], padding='VALID', bval=None, activation_fn=tf.nn.relu)
        with tf.variable_scope("3x3_3"):
            # still not sure about strides
            a = conv(a, depth = 64, ksize=[3,3], strides=[2,2], padding='SAME', bval=None, activation_fn=tf.nn.relu)
        with tf.variable_scope("3x3_maxpool_branch_1a"):
            aa = maxpool(a, ksize=[3,3], strides=[2,2], padding='VALID')
        with tf.variable_scope("3x3_conv_branch_1b"):
            bb = conv(a, depth=96, ksize=[3,3], strides=[2,2], padding='VALID',bval=None, activation_fn=tf.nn.relu)
        with tf.variable_scope("concat_1"):
            a = tf.concat([aa,bb],-1)
        with tf.variable_scope("branch_2a"):
            # here assuming stride 1 since 1x1 conv ?? also this is not an inception block so has activation????
            aa = conv(a, depth=64, ksize=[1,1], strides=[1,1], padding='SAME', activation_fn=tf.nn.relu)
            # assuming stride 2 tho not sure
            aa = conv(aa, depth=96, ksize=[3,3], strides=[2,2], padding='VALID', activation_fn=tf.nn.relu)
        with tf.variable_scope("branch_2b"):
            bb = conv(a, depth=64, ksize=[1,1], strides=[1,1], padding='SAME', bval=None, actication_fn=tf.nn.relu)
            bb = conv(bb, depth=64, ksize=[7,1], strides=[1,1], padding='SAME', bval=None, activation_fn=tf.nn.relu)
            bb = conv(bb, depth=64, ksize=[1,7], strides=[1,1], padding='SAME', bval=None, activation_fn=tf.nn.relu)
            bb = conv(bb, depth=96, ksize=[3,3], strides=[2,2], padding='VALID', bval=None, activation_fn=tf.nn.relu)
        with tf.variable_scope("concat_2"):
            a = tf.concat([aa,bb],axis= -1)
        with tf.variable_scope("branch_3a"):
            aa = conv(a, depth=192, ksize=[3,3],strides=[2,2], padding='VALID', bval=None, activation_fn=tf.nn.relu)
            

In [18]:
# Inception-Resnet-v2 Overall Structure
def inception_resnet_v2_full(inputs,keep_prob):
    input_depth - inputs.get_shape()[-1]
    with tf.variable_scope('stem'):
        flowing_data = inception_stem(inputs)
    with tf.variable_scope('inception_resnet_A_x5'):
        for i in range(5)
            flowing_data = inception_renet_A(flowing_data)
    with tf.variable_scope('reduction_A'):
        flowing_data = reduction_A(flowing_data)
    with tf.variable_scope('inception_resnet_B_x10'):
        for i in range(10):
            flowing_data = inception_resnet_B(flowing_data)
    with tf.variable_scope('reduction_B'):
        flowing_data = reduction_B(flowing_data)
    with tf.variable_scope('inception_resnet_C_x5'):
        for i in range(5):
            flowing_data = inception_resnet_C(flowing_data)
    with tf.variable_scope('avg_pool'):
        flowing_data = avgpool(inputs, ksize, strides, padding='VALID', name=None)
    with tf.variable_scope('dropout'):
        flowing_data = tf.nn.dropout(flowing_data)
    with tf.variable_scope('softmax'):
        flowing_data = tf.nn.softmax(flowing_data)
    return flowing_data

SyntaxError: invalid syntax (<ipython-input-18-3fb0b42abd79>, line 7)